### FM (Factorization Machines)과 MF (Matrix Factorization) 설명 

#### FM (Factorization Machines)  
  
FM은 선형 모델의 확장으로, 고차원의 데이터에서 상호 작용을 고려하여 모델링할 수 있습니다.  
주로 특징 간의 상호 작용을 고려하여 예측을 수행하는 데 사용됩니다.  
FM은 일반적으로 희소한 데이터에 효과적이며, 특징 간의 관계를 학습하기 위해 사용됩니다.  
FM은 높은 차원의 희소 데이터에 적합하며, 예를 들어 사용자와 상품 간의 상호 작용을 고려할 때 유용합니다.  

#### MF (Matrix Factorization)  
  
MF는 행렬 분해를 기반으로 하여 사용자와 상품 간의 관계를 학습하는 데 사용됩니다.  
주로 사용자-상품 평점 행렬과 같은 행렬 데이터를 분해하여 사용합니다.  
MF는 사용자와 상품을 잠재적인 요인 공간(latent factor space)에 매핑하여 사용자와 상품 간의 상호 작용을 모델링합니다.  
MF는 주로 사용자-상품 평가 예측에 사용되며, 추천 시스템에서 널리 사용됩니다.  


#### 공통점  
각각의 변수를 f차원의 latent Factor로 매핑하여 변수간의 상관관계를 Latent Factor의 내적으로 모델링 
  
#### 차이점
mf는 user, item, rating 만을 사용. 반면 fm은 다양한 feature를 concatnate하여 사용  
mf는 user * item의 내적으로 rating을 계산, 여기서 식은 user*item=rating  
반면 fm은 하나의 특성 벡터로 치환, user|item|기타feature1=rating 즉 다항회귀와 유사
fm은 sparse data에서도 파라메터 추정 가능  
fm은 변수간의 상호작용을 고려하여 예측 가능  


In [20]:
#FM (Factorization Machines)
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
#from fastFM import als

# 훈련 데이터 생성 (예시)
# 데이터 로드 및 설정
df = pd.read_csv('dataset/ratings.csv')


data = ''
'''
df를 [{column1:row1_value1,column2:row1_value2},
    {column1:row2_value1,column2:row2_value2},
    ...
    의 형태로 변환
]
'''

# DictVectorizer를 사용하여 데이터를 희소 행렬로 변환
v = DictVectorizer() 
X = v.fit_transform(data)

# 타겟 값 추출
y = [d['rating'] for d in df]
print(y)
'''
# 훈련 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# FM 모델 훈련
fm = als.FMClassification(n_iter=1000, init_stdev=0.1, rank=10, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)

# 예측 수행
y_pred = fm.predict(X_test)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print("FM 정확도:", accuracy)'''

AttributeError: 'str' object has no attribute 'items'

In [21]:
# Matrix Factorization (MF) 구현
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd
from surprise.model_selection import GridSearchCV


# 데이터 로드 및 설정
df = pd.read_csv('dataset/ratings.csv')

#userId,movieId,rating
reader = Reader(line_format='user item rating', sep=',', rating_scale=(0, 1))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# 훈련 및 테스트 데이터로 분할
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# MF 모델 훈련
model = SVD()
model.fit()

# 예측 수행
predictions = model.test()

# 정확도 평가
accuracy_mf = accuracy.rmse(predictions)
print("MF 정확도:", accuracy_mf)

TypeError: fit() takes exactly 2 positional arguments (1 given)

In [22]:
from surprise.model_selection import GridSearchCV

# SVD 모델의 매개변수 그리드 설정
param_grid = {'n_factors': [1, 5], 'n_epochs': [1, 3], 'lr_all': [0.005, 0.01, 0.02], 'reg_all': [0.02, 0.05, 0.1]}

# 그리드 서치를 사용하여 최적의 매개변수 찾기
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=20)
gs.fit()

# 최적의 매개변수 출력
print("최적의 매개변수:", gs.best_params['rmse'])

# 최적의 매개변수로 모델 초기화 및 훈련
best_model = gs.best_estimator['rmse']
best_model.fit()

# 테스트 데이터에 대한 예측 수행
predictions = best_model.test()

# 정확도 평가
accuracy_mf = accuracy.rmse(predictions)
print("MF 정확도:", accuracy_mf)


TypeError: fit() missing 1 required positional argument: 'data'